In [1]:
# %% Package imports
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from deep_audio import Audio, Visualization, Directory, Model, JSON
import numpy as np
from joblib import Parallel, delayed
import multiprocessing
import joblib

In [2]:
# %% Load dataset
sampling_rate = 22050

method_algo = 'mfcc'

x, y, mapping = Directory.load_json_data(f'processed/{method_algo}/{method_algo}_{sampling_rate}.json',
                                         inputs_fieldname=method_algo)

In [3]:
# SPLIT 5 FOLDS

random_state = 42

kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
kf.get_n_splits(x)

x_holder = []

for row in x:
    x_holder.append(row.flatten())

x = np.array(x_holder)

n = len(x)

x = x[:n]
y = y[:n]

In [4]:
def process_fold(train_index, test_index):
    x_train, x_val = x[train_index], x[test_index]
    y_train, y_val = y[train_index], y[test_index]
    
    model = svm.SVC(C=10, kernel='rbf', decision_function_shape='ovo')
        
    model.fit(x_train, y_train)
    
    # TESTA ACCURÁCIAS
    
    score_test = model.score(x_val, y_val)

    score_train = model.score(x_train, y_train)

    y_hat = model.predict(x_val)
    
    # SALVA MODELO
    
    filename = f'model/svm/5fold/{method_algo}_{sampling_rate}/acc{score_test}_seed{random_state}.sav'
    
    Directory.create_directory(filename, file=True)
    
    joblib.dump(model, filename)
    
    # SALVA ACURÁCIAS E PARAMETROS
    
    return {
        'seed': random_state,
        'feature_method': method_algo,
        'sample_rate': sampling_rate,
        'train_test': [len(x_train), len(x_val)],
        'score_train': score_train,
        'score_test': score_test,
        'f1_micro': f1_score(y_hat, y_val, average='micro'),
        'f1_macro': f1_score(y_hat, y_val, average='macro'),
        'model_file': f'acc{score_test}_seed{random_state}.sav',
        'params': model.get_params()
    }

In [ ]:
num_cores = multiprocessing.cpu_count()

dump_info = Parallel(n_jobs=num_cores, verbose=3)(
    delayed(process_fold)(train_index, test_index) for train_index, test_index in kf.split(x))

JSON.create_json_file(f'model/svm/5fold/{method_algo}_{sampling_rate}/info.json', dump_info)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


In [ ]:
#     data = {
#         "mapping": mapping.tolist(),
#         "labels": y_val.tolist(),
#         "mfcc": x_val.tolist(),
#     }

#     JSON.create_json_file(f'datatest/svm/{method_algo}_{sampling_rate}/datatest_{random_state}_{x.shape[0]}.json', data)

In [ ]:
#     param_grid = {
#         'C': [0.1, 1, 10, 100, 1000],
#     #     'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
#         'kernel': ['linear', 'rbf'],
#         'decision_function_shape': ['ovo', 'ovr']
#     }

    # %% training

    # model = RandomForestClassifier()

    # model = GridSearchCV(svm.SVC(), param_grid, refit=True, verbose=3, n_jobs=8)

In [ ]:
# # %%

# # # print best parameter after tuning
# print(model.best_params_)
# #
# # # print how our model looks after hyper-parameter tuning
# print(model.best_estimator_)

# grid_predictions = model.predict(x_val)
# #
# # # print classification report
# print(classification_report(y_val, grid_predictions))